# Importing Essential Modules and Packages

In [1]:
import os
import pandas as pd
import librosa
import glob
import librosa.display
import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils

from sklearn import metrics
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from keras.callbacks import EarlyStopping

from keras import regularizers

from sklearn.preprocessing import LabelEncoder

from datetime import datetime


ModuleNotFoundError: No module named 'librosa'

# Preparing the Data


In [2]:
filelist = os.listdir("drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat")
cat_files = pd.DataFrame(filelist)
metadata = pd.read_csv("drive/MyDrive/ProjectMGN/CatandDog/train_test_split.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat'

In [3]:
filelist = os.listdir("drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog")
dog_files = pd.DataFrame(filelist)

FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog'

In [4]:
cat_files.head()

NameError: name 'cat_files' is not defined

In [5]:
dog_files.head()

NameError: name 'dog_files' is not defined

In [6]:
metadata.head()

NameError: name 'metadata' is not defined

In [7]:
cat_files

NameError: name 'cat_files' is not defined

# Merge the Cat Dog Dataset and Shuffling the Dataset

In [8]:
cat_files['label']='0'
dog_files['label']='1'
df = pd.concat([dog_files,cat_files],ignore_index=True)
df = df.rename(columns={0:'file'})
df = df.sample(frac=1).reset_index(drop=True)
df.head()

NameError: name 'cat_files' is not defined

In [9]:
# Checking Class Balance

df['label'].value_counts()

NameError: name 'df' is not defined

In [10]:
# Checking for Erroneous Folders

df[df['file']=='.DS_Store']

NameError: name 'df' is not defined

# Function to Extract Primary Features from Audio File

In [11]:
def extract_features(data):
  # file_name - Directory to cat/dog audio files
  try:
    file_name = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat/'+str(data.file)
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  except FileNotFoundError:
    file_name = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog/'+str(data.file)
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  # print(file_name)
  mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
  stft = np.abs(librosa.stft(X))
  chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
  contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
  tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
  label = data.label
  return mfccs, chroma, mel, contrast, tonnetz, label


# Generating Features


In [12]:
startTime=datetime.now()
features_labels = df.apply(extract_features, axis=1)
endtime=datetime.now()
print("Total Time Taken:",endtime-startTime)

NameError: name 'datetime' is not defined

In [13]:
# Checking Feature Labels

features_labels

NameError: name 'features_labels' is not defined

In [14]:
print(f"len(feautures_labels):{len(features_labels)} and len(features_labels[0]):{len(features_labels[0])}")

NameError: name 'features_labels' is not defined

# Saving the Feature Vector Generated

In [15]:
np.save('drive/MyDrive/ProjectMGN/features_label_catdog_max_dataset',features_labels)

NameError: name 'np' is not defined

In [16]:
features_labels=np.load('drive/MyDrive/ProjectMGN/features_label_catdog_max_dataset.npy', allow_pickle=True)

NameError: name 'np' is not defined

# Inputs for ANN

# Feature Vector

In [17]:
features=[]
for i in range(0, len(features_labels)):
    features.append(np.concatenate((features_labels[i][0], features_labels[i][1], 
                features_labels[i][2], features_labels[i][3],
                features_labels[i][4]), axis=0))

NameError: name 'features_labels' is not defined

In [18]:
print(f"len(feautures):{len(features)} and len(features[0]):{len(features[0])}")

IndexError: list index out of range

In [19]:
X = np.array(features)
y = np.array(df['label'])

NameError: name 'np' is not defined

In [20]:
# OneHotEncoding Labels

lb = LabelEncoder()
y = to_categorical(lb.fit_transform(y))

NameError: name 'LabelEncoder' is not defined

In [21]:
X.shape

NameError: name 'X' is not defined

In [22]:
y.shape

NameError: name 'y' is not defined

# Normalizing the Data

In [23]:
ss = StandardScaler()
TEST_LIMIT = 250
x_train = X[:TEST_LIMIT]
y_train = y[:TEST_LIMIT]
x_test = X[TEST_LIMIT:]
y_test = y[TEST_LIMIT:]
X_train=ss.fit_transform(x_train)
X_test=ss.transform(x_test)

NameError: name 'StandardScaler' is not defined

# Making and evaluating the ANN Model

In [24]:
# Build a simple dense model with early stopping with softmax for categorical classification
# Total of 2 Classes - 0 representing Cat and representing  Dog

model = Sequential()

model.add(Dense(193, input_shape=(193,), activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics='accuracy')

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

NameError: name 'Sequential' is not defined

In [25]:
history = model.fit(X_train, y_train, batch_size=256, epochs=100, validation_data=(X_test, y_test))

NameError: name 'model' is not defined

In [26]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Set figure size.
plt.figure(figsize=(12, 8))

# Generate line plot of training, testing loss over epochs.
plt.plot(train_accuracy, label='Training Accuracy', color='#185fad')
plt.plot(val_accuracy, label='Validation Accuracy', color='orange')

# Set title
plt.title('Training and Validation Accuracy by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Categorical Crossentropy', fontsize = 18)
plt.xticks(range(0,100,5), range(0,100,5))
plt.legend(fontsize = 18);

NameError: name 'history' is not defined

In [ ]:
# Predicting On Test Data

predict_x=model.predict(X_test) 
preds=np.argmax(predict_x,axis=1)
preds = lb.inverse_transform(preds)
df_test = df[TEST_LIMIT:]
df_test['preds']=preds